In [1]:
import os
import json
import requests
import codecs
import tabula


In [10]:
ok_statuses = [200, 201, 202]

url_province_latest = "https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-province/dpc-covid19-ita-province-latest.csv"

data_file_path = os.path.join("..","data")
prov_data_file_name = "virus-province.csv"
prov_data_file = os.path.join(data_file_path, prov_data_file_name)

In [15]:
result = requests.get(url_province_latest)
if result.status_code not in ok_statuses:
    assert "Get data failed. Received error code: {er}".format(er=str(result.status_code))
    
with codecs.open(prov_data_file, "w", "utf-8") as fh:
    fh.write(result.text)


In [16]:
url_region_pdf = "http://www.salute.gov.it/imgs/C_17_notizie_4362_0_file.pdf"
result = requests.get(url_region_pdf)
if result.status_code not in ok_statuses:
    assert "Get data failed. Received error code: {er}".format(er=str(result.status_code))

#region_data_file = os.path.join(os.sep,"tmp","region.pdf")    
with open(prov_data_file, "wb") as fh:
    fh.write(result.content)


In [2]:
#declare the path of your file
file_path = "/home/giovanni/Downloads/C_17_notizie_4362_0_file.pdf"#Convert your file
df = tabula.read_pdf(file_path)

'pages' argument isn't specified.Will extract only from page 1 by default.
Got stderr: Mar 31, 2020 6:11:18 PM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font CIDFont+F1 are not implemented in PDFBox and will be ignored
Mar 31, 2020 6:11:18 PM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font CIDFont+F2 are not implemented in PDFBox and will be ignored



In [7]:
df_ita = df[1]

In [8]:
df_region = df[0]